  # ДЗ №4. Траснформеры

In [28]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import torch
import time
import numpy as np
import pandas as pd
import random

from tqdm import tqdm
from transformers import AdamW, BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch import nn
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader
from math import log2, ceil
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

## EDA

Загрузим данные и разделим их на тренировочный, тестовый и валидационный наборы.

In [29]:
_data_train_val = pd.read_csv('./data/in_domain_train.csv')
_X_train, _X_val, _y_train, _y_val = train_test_split(_data_train_val['sentence'], _data_train_val['acceptable'], test_size=0.1, random_state=123)

_X_train = _X_train.to_numpy()
_X_val = _X_val.to_numpy()
_y_train = _y_train.to_numpy()
_y_val = _y_val.to_numpy()

_data_test = pd.read_csv('./data/in_domain_dev.csv')
_X_test = _data_test['sentence']
_y_test = _data_test['acceptable']

Посмотрим на баланс классов.

In [30]:
#_y_train.hist()

Классы несбалансированы. Для оценки качества будем использовать MCC (Matthews Correlation Coefficient).

## BERT

### Обучение

Определим несколько утлитных классов.

In [31]:
class RuBertTokenizer():
    '''
    Принимает корпус текстов, токенизирует его и возвращает input_ids и attemtion_masks в виде тензоров.
    '''
    def __init__(self):
        # https://huggingface.co/ai-forever/ruBert-base
        self._bert_tokenizer = BertTokenizer.from_pretrained('ai-forever/ruBert-base')

    def transform(self, X):
        max_length = self._get_max_document_length(X, self._bert_tokenizer)

        input_ids = []
        attention_masks = []

        for document in X:
            encoded_dict = self._bert_tokenizer.encode_plus(document,
                                                            add_special_tokens=True,
                                                            max_length=max_length,
                                                            pad_to_max_length=True,
                                                            return_tensors='pt',
                                                            truncation=True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        # преобразуем в тензоры
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return input_ids, attention_masks
    
    def _get_max_document_length(self, X, tokenizer:BertTokenizer):
        # находим максимальную длину документа
        max_length = 0
        for document in X:
            tokenized_document = tokenizer.encode(document, add_special_tokens=True)
            max_length = max(max_length, len(tokenized_document))

        # Увеличиваем длину до ближайшей степени двойки.
        # Например, если максимальная длина документа 41, то берем 64.
        result = pow(2, ceil(log2(max_length)))

        if max_length != result:
            print(f'Максимальная длинна документа [{max_length}] уже является степенью двойки.')
        else:
            print(f'Максимальная длинна документа [{max_length}]. Берем [{result}].')

        return result

In [32]:
class Trainer:
    '''
    Утилитный класс для обучения и валидации модели.
    '''

    def __init__(self, model, optimizer, num_of_epochs:int):
        self._model = model
        self._optimizer = optimizer
        self._num_of_epochs = num_of_epochs

    def train(self, data_loader:DataLoader):
        self._model.train() # переводим модель в режим обучения

        total_loss = 0

        start_time = time.time()

        for epoch in range(self._num_of_epochs):
            print(f'Эпоха {epoch}...')
            for batch in tqdm(data_loader):
                # обнуляем предыдущие значения градиентов
                self._model.zero_grad()

                input_ids = batch[0]
                attention_mask = batch[1]
                labels = batch[2]

                # делаем предсказание
                # сравниваем предсказанные значения с истинными
                pred = self._model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = pred.loss

                total_loss += loss.item()

                # вычисляем градиент функции потерь
                loss.backward()
                
                # обновляем веса модели
                self._optimizer.step()

        elapsed_time = time.time() - start_time
        
        avergate_loss = total_loss / len(data_loader)
        print(f'Средняя ошибка: {avergate_loss}')
        print(f'Время обучения: {datetime.timedelta(elapsed_time)}')
    
    def test(self, data_loader:DataLoader):
        self._model.eval() # переводим модель в режим использования

        total_loss = 0
        y_pred = []

        softmax = nn.Softmax(dim=1)

        for batch in tqdm(data_loader):
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]

            with self._model.no_grad():
                pred = self._model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                total_loss += pred.loss.item()
                probabilities = softmax(pred.logits)
                y_pred.append(0 if probabilities[0] > 0.5 else 1)

        avergate_loss = total_loss / len(data_loader)
        print(f'Средняя ошибка: {avergate_loss}')
        print(f'MCC: {matthews_corrcoef(labels, y_pred)}')

Загрузим модель.

In [33]:
_bert_model = BertForSequenceClassification.from_pretrained('ai-forever/ruBert-base',
                                                            num_labels = 2,
                                                            output_attentions = False,
                                                            output_hidden_states = False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Токенизируем корпус и обучим модель.

In [34]:
def prepare_dataset(X, y):
    input_ids, attention_masks = RuBertTokenizer().transform(X)
    tensor_y = torch.tensor(y)
    tensor_dataset = TensorDataset(input_ids, attention_masks, tensor_y)
    return DataLoader(dataset=tensor_dataset, batch_size=64, shuffle=True)

_train_data_loader = prepare_dataset(_X_train, _y_train)
_val_data_loader = prepare_dataset(_X_val, _y_val)
_test_data_loader = prepare_dataset(_X_test, _y_test)

_optimizer = AdamW(_bert_model.parameters())
_trainer = Trainer(_bert_model, _optimizer, num_of_epochs=1)

_trainer.train(_train_data_loader)

Максимальная длинна документа [45] уже является степенью двойки.
Максимальная длинна документа [42] уже является степенью двойки.
Максимальная длинна документа [41] уже является степенью двойки.
Эпоха 0...


  2%|▏         | 2/111 [00:37<33:51, 18.64s/it]


KeyboardInterrupt: 

## Few-/zero-shot с GPT3

## RuT5

### Обучение

### Тестирование